In [190]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import sparse

### Ler Dados

In [4]:
preg = []

with open('licitacoes_ultimas.json', 'r') as f:
    e = f.readline()
    while e:
        preg.append(json.loads(e))
        e = f.readline()

df = pd.DataFrame(preg)

### Ajuste dados

In [5]:
df['municipio'] = df['municipio'].apply(lambda s: s.get('_id', None).get('$oid', None) if type(s) == dict else None)

for i in ['dataCaptura', 'dataInsercao', 'dataClassificacao', 'dataPublicacao',	'dataInicialProposta',	'dataFinalProposta']:
    df[i] = df[i].apply(lambda s: s.get('$date', None) if type(s) == dict else None)

df['_id'] = df['_id'].apply(lambda s: s.get('$oid', None) if type(s) == dict else None)

In [6]:
df.head()

,_id,idLicitacao,portal,orgao,orgaoSemAcento,unidadeGestora,url,objeto,objetoSemAcento,processo,...,anexos,dataCaptura,dataInsercao,classificado,interno,municipio,dataClassificacao,exclusivoMPE,linhasDeFornecimento,valorDeReferenciaGlobal
0,60c02f7b3a27845d7a1233eb,5b019bde85cd436d5c272d08b4e1728847dc0165647890...,BBMNet,PREFEITURA MUNICIPAL DE AMERICANA,PREFEITURA MUNICIPAL DE AMERICANA,PREFEITURA MUNICIPAL DE AMERICANA / PREFEITURA...,https://effecti.com.br/content/redirect.html?url=,AQUISIÇÃO DE EQUIPAMENTOS ODONTOLÓGICOS PARA O...,AQUISICAO DE EQUIPAMENTOS ODONTOLOGICOS PARA O...,0762021,...,[{'nome': 'Edital 076.2021 Prefeitura Municip...,2021-06-08T18:40:46Z,2021-06-09T03:03:23.394Z,False,False,5fd1428573f1d771dc5bf232,None,NaN,NaN,NaN
1,60c02f89d11ca15d9461f1e0,f892dd76a979cba553606ac28a42c24414020600109c51...,BBMNet,PREFEITURA MUNICIPAL DE TATUÍ,PREFEITURA MUNICIPAL DE TATUi,PREFEITURA MUNICIPAL DE TATUÍ / 46.634.564/000...,https://effecti.com.br/content/redirect.html?url=,PREGÃO ELETRÔNICO DE REGISTRO DE PREÇOS PARA A...,PREGAO ELETRONICO DE REGISTRO DE PRECOS PARA A...,482021,...,[{'nome': '=?Windows-1252?Q?Edital_48.2021__Pr...,2021-06-08T18:33:35Z,2021-06-09T03:03:37.099Z,True,False,5fd1428573f1d771dc5bf03c,2021-06-09T15:14:39.837Z,False,[11049],0.0
2,60c02edf3a27845d7a12339c,d62e6eeda9932e1c9b6f08e05de2b64c534acb08b115b7...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,MATERIAL DE CONSUMO,380261000012021OC00119,...,[],2021-06-08T18:32:36Z,2021-06-09T03:00:47.129Z,False,False,5fd1428573f1d771dc5beff3,None,NaN,NaN,NaN
3,60c02ee0d11ca15d9461f18d,73ba27b604d51f03d68e1d92bb2e063a79df8ad392744e...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,MATERIAL DE CONSUMO,380256000012021OC00072,...,[],2021-06-08T18:32:01Z,2021-06-09T03:00:48.091Z,False,False,5fd1428573f1d771dc5bf1e2,None,NaN,NaN,NaN
4,60c02f384ef4335d809f1298,70f63399f4acbeb2ce71f8d724a55bb949fa78e69e1a9b...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,MATERIAL DE CONSUMO,380251000012021OC00065,...,[],2021-06-08T18:31:25Z,2021-06-09T03:02:16.372Z,False,False,5fd1428573f1d771dc5bef8d,None,NaN,NaN,NaN


### Expandir itens

In [7]:
itens = df['itensEdital'].apply(pd.Series)

itens = itens.applymap(lambda s: s.get('produtoLicitado', None) if type(s) == dict else None)

itens = itens.stack().rename('item').to_frame().droplevel(1)

In [8]:
df_itens = pd.merge(
    itens, 
    df.drop(['itensEdital'], axis = 1), 
    left_index = True, 
    right_index = True
).reset_index()

In [9]:
df_itens.head(3)

,index,item,_id,idLicitacao,portal,orgao,orgaoSemAcento,unidadeGestora,url,objeto,...,anexos,dataCaptura,dataInsercao,classificado,interno,municipio,dataClassificacao,exclusivoMPE,linhasDeFornecimento,valorDeReferenciaGlobal
0,2,"PAPEL HIGIENICO - FOLHA DUPLA, CLASSE 01, NEUT...",60c02edf3a27845d7a12339c,d62e6eeda9932e1c9b6f08e05de2b64c534acb08b115b7...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,...,[],2021-06-08T18:32:36Z,2021-06-09T03:00:47.129Z,False,False,5fd1428573f1d771dc5beff3,None,NaN,NaN,NaN
1,2,TOALHA DE PAPEL-SIMPLES-INTERFOLHADA INSTITUCI...,60c02edf3a27845d7a12339c,d62e6eeda9932e1c9b6f08e05de2b64c534acb08b115b7...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,...,[],2021-06-08T18:32:36Z,2021-06-09T03:00:47.129Z,False,False,5fd1428573f1d771dc5beff3,None,NaN,NaN,NaN
2,2,"SCANNER DE IMPRESSAO DIGITAL, PARA RECONHECIME...",60c02edf3a27845d7a12339c,d62e6eeda9932e1c9b6f08e05de2b64c534acb08b115b7...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,SECRETARIA ADMINISTRACAO PENITENCIARIA - PENIT...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,...,[],2021-06-08T18:32:36Z,2021-06-09T03:00:47.129Z,False,False,5fd1428573f1d771dc5beff3,None,NaN,NaN,NaN


Formatoo dados

In [10]:
print(df_itens.shape)

(35936, 28)


#### TFIDF

In [225]:
stop_words = set(nltk.corpus.stopwords.words('portuguese'))

tfidf = TfidfVectorizer(max_features = 5000, ngram_range = (0, 3), stop_words = stop_words)

tf_result = tfidf.fit_transform(df_itens['item'])

#### Selecionar Variáveis

In [226]:
# Selecionar modalidades
modalidade = pd.get_dummies(df_itens['modalidade'])

modalidade = modalidade[modalidade.sum().loc[lambda s: s > 100].index]

# Selecionar municipios
municipios = pd.get_dummies(df_itens['municipio'])

municipios = municipios[municipios.sum().loc[lambda s: s > 20].index]

# Selecionar MPE
mpe = pd.get_dummies(df_itens['exclusivoMPE'])

# Selecionar Portal de listagem
portal = pd.get_dummies(df_itens['portal'])

portal = portal[portal.sum().loc[lambda s: s > 100].index]

# Selecionar orgao
orgao = pd.get_dummies(df_itens['orgao'])

orgao = orgao[orgao.sum().loc[lambda s: s > 10].index]

### Unir dados

In [269]:
features = pd.concat(
    [
        modalidade, 
#         municipios,
        mpe,
        portal,
#         orgao,
    ], axis = 1
)

t = sparse.hstack([tf_result, sparse.csr_matrix(features)], format = 'csr')

### Teste 1 itens aleatório

Seleção Itens

In [282]:
sample = df_itens.sample(1)

sample[['item', 'orgao', 'portal']]

,item,orgao,portal
31442,"TINTA BASE ÁGUA - TINTA BASE ÁGUA, COMPOSIÇÃO ...",PREFEITURA MUNICIPAL DE ASSIS - SÃO PAULO,ComprasNet - Pregão


Itens sugeridos

In [283]:
similarity = cosine_similarity(t, t[sample.index])

df_itens.iloc[similarity.reshape(-1).argsort()[::-1]][['item', 'orgao', 'portal']][1:21]

,item,orgao,portal
21764,"TINTA ACRÍLICA - TINTA ACRÍLICA, COMPONENTES L...",PREFEITURA MUNICIPAL DE ITAQUAQUECETUBA/SP,ComprasNet - Pregão
21763,"TINTA ACRÍLICA - TINTA ACRÍLICA, COMPONENTES L...",PREFEITURA MUNICIPAL DE ITAQUAQUECETUBA/SP,ComprasNet - Pregão
13167,"ALGICIDA - ALGICIDA, COMPOSIÇÃO POLIQUATERNÁRI...",ESCOLA PREPARATÓRIA DE CADETES DO EXÉRCITO,ComprasNet - Pregão
31445,"TINTA ACRÍLICA - TINTA ACRÍLICA, COMPONENTES Á...",PREFEITURA MUNICIPAL DE ASSIS - SÃO PAULO,ComprasNet - Pregão
31447,"TINTA ACRÍLICA - TINTA ACRÍLICA, COMPONENTES Á...",PREFEITURA MUNICIPAL DE ASSIS - SÃO PAULO,ComprasNet - Pregão
13168,"ALGICIDA - ALGICIDA, COMPOSIÇÃO POLIQUATERNÁRI...",ESCOLA PREPARATÓRIA DE CADETES DO EXÉRCITO,ComprasNet - Pregão
13164,"CLARIFICANTE - CLARIFICANTE, COMPOSIÇÃO POLICL...",ESCOLA PREPARATÓRIA DE CADETES DO EXÉRCITO,ComprasNet - Pregão
31444,"TINTA ACRÍLICA - TINTA ACRÍLICA, COMPONENTES R...",PREFEITURA MUNICIPAL DE ASSIS - SÃO PAULO,ComprasNet - Pregão
31443,"TINTA ACRÍLICA - TINTA ACRÍLICA, COMPONENTES R...",PREFEITURA MUNICIPAL DE ASSIS - SÃO PAULO,ComprasNet - Pregão
31431,"TINTA ACRÍLICA - TINTA ACRÍLICA, COMPONENTES R...",PREFEITURA MUNICIPAL DE ASSIS - SÃO PAULO,ComprasNet - Pregão


### Teste #1 itens selecionados

Itens selecionados

In [284]:
sample = df_itens.iloc[[28039, 12599]]

sample

,index,item,_id,idLicitacao,portal,orgao,orgaoSemAcento,unidadeGestora,url,objeto,...,anexos,dataCaptura,dataInsercao,classificado,interno,municipio,dataClassificacao,exclusivoMPE,linhasDeFornecimento,valorDeReferenciaGlobal
28039,4139,Ácido Ascórbico 500 mg,60af0b86493beb4f89839d67,9c93147357ffb1b05f59a3a0fe18fbf47cd680c227ab2f...,Licitações-e,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,https://effecti.com.br/content/redirect.html?u...,REGISTRO DE PREÇO PARA AQUISIÇÃO PARCELADA DE ...,...,"[{'nome': 'MEDICAMENTO.PDF', 'url': 'https://e...",2021-05-25T19:02:40Z,2021-05-27T03:01:26.615Z,True,False,5fd1428573f1d771dc5bf2e3,2021-05-27T17:24:19.508Z,False,[10806],0.0
12599,1564,ÁCIDO GRAXO ESSENCIAL PARA FERIDAS,60b84689d2fbf92b20e4f8eb,7d37d20e940855f96b9305e61bcb956c3ecea76df1e7b7...,Prefeitura de Diadema - SP,PREFEITURA DO MUNICIPIO DE DIADEMA,PREFEITURA DO MUNICIPIO DE DIADEMA,PREFEITURA DO MUNICIPIO DE DIADEMA,https://effecti.com.br/content/redirect.html?u...,REGISTRO DE PREÇOS PARA O FORNECIMENTO DE MATE...,...,[{'nome': 'pc081- Fornecimento Material de Enf...,2021-06-02T15:18:46Z,2021-06-03T03:03:37.375Z,True,False,5fd1428573f1d771dc5bf210,2021-06-03T14:13:42.769Z,False,[11035],0.0


Sugestões

In [285]:
similarity = cosine_similarity(t, t[sample.index].mean(axis = 0))

df_itens.iloc[similarity.reshape(-1).argsort()[::-1]][['item', 'orgao', 'portal']][:20]

,item,orgao,portal
12567,ÁCIDO GRAXO ESSENCIAL PARA FERIDAS,PREFEITURA DO MUNICIPIO DE DIADEMA,Prefeitura de Diadema - SP
28039,Ácido Ascórbico 500 mg,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,Licitações-e
12599,ÁCIDO GRAXO ESSENCIAL PARA FERIDAS,PREFEITURA DO MUNICIPIO DE DIADEMA,Prefeitura de Diadema - SP
14755,ÁCIDO ÚRICO,MUNICIPIO DE SANTOS /(2) SECRETARIA MUNICIPAL ...,Licitações-e
28040,CRI - Ácido Ascórbico 500 mg,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,Licitações-e
14914,ÁCIDO ACETILSALICÍLICO 100 MG,PREFEITURA DO MUNICIPIO DE DIADEMA,Prefeitura de Diadema - SP
14955,ÁCIDO ACETILSALICÍLICO 100 MG,PREFEITURA DO MUNICIPIO DE DIADEMA,Prefeitura de Diadema - SP
28035,Ácido Acetilsalicilico 100mg comprimido,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,Licitações-e
28051,Ácido Valproico 500mg comprimido,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,Licitações-e
28047,Ácido Valpróico 250mg,PREFEITURA MUNICIPAL DE ITIRAPUA /(1) PREFEITU...,Licitações-e


### Teste #2 itens selecionados

Itens Selecionados

In [286]:
sample = df_itens.iloc[[918, 22901, 21401, 21403]]

sample

,index,item,_id,idLicitacao,portal,orgao,orgaoSemAcento,unidadeGestora,url,objeto,...,anexos,dataCaptura,dataInsercao,classificado,interno,municipio,dataClassificacao,exclusivoMPE,linhasDeFornecimento,valorDeReferenciaGlobal
918,148,"PASTILHA DE FREIO PARA MOTO, POSICAO TRASEIRA,...",60c02f91fbf9575d9a0c91f7,9fb456c93d682b741be7acdcd1b9b5dd740a7b656bce7e...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA DA SEGURANCA PUBLICA COM.POLIC.AREA...,SECRETARIA DA SEGURANCA PUBLICA COM.POLIC.AREA...,SECRETARIA DA SEGURANCA PUBLICA COM.POLIC.AREA...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,...,[{'nome': '180179000012021oc00444edital0806202...,2021-06-08T18:04:28Z,2021-06-09T03:03:45.016Z,True,False,5fd1428573f1d771dc5bf20f,2021-06-09T11:39:55.78Z,False,[10850],0.0
22901,3152,"CABO DE EMBREAGEM PARA MOTO, PARA MOTOS DA MAR...",60b1af02d2fbf92b20e4f6db,c67a1a882e64e7fc7d7943ce14e8a437f1aa1f83bd5cd3...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA DA SEGURANCA PUBLICA - CTO. SUPRIM....,SECRETARIA DA SEGURANCA PUBLICA - CTO. SUPRIM....,SECRETARIA DA SEGURANCA PUBLICA - CTO. SUPRIM....,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,...,[],2021-05-28T00:46:52Z,2021-05-29T03:03:30.454Z,True,False,5fd1428573f1d771dc5bf212,2021-06-01T20:19:09.873Z,True,[10850],0.0
21401,2993,"VENTOINHA DO RADIADOR PARA MOTO, MARCA MOTOS D...",60b1ae86e1003d2b401a822a,f7c2af880d254307ceb63619f5dcdcc7740ced53cabef6...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,...,[],2021-05-28T13:21:10Z,2021-05-29T03:01:26.99Z,True,False,5fd1428573f1d771dc5bf264,2021-06-01T19:46:53.09Z,True,[10797],0.0
21403,2993,"KIT RELAÇÃO/TRANSMISSAO PARA MOTO, MARCA HONDA...",60b1ae86e1003d2b401a822a,f7c2af880d254307ceb63619f5dcdcc7740ced53cabef6...,BEC / Imprensa Oficial / Pregão SP,SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,https://effecti.com.br/content/redirect.html?u...,MATERIAL DE CONSUMO,...,[],2021-05-28T13:21:10Z,2021-05-29T03:01:26.99Z,True,False,5fd1428573f1d771dc5bf264,2021-06-01T19:46:53.09Z,True,[10797],0.0


Sugestões

In [287]:
similarity = cosine_similarity(t, t[sample.index].mean(axis = 0))

df_itens.iloc[similarity.reshape(-1).argsort()[::-1]][['item', 'orgao', 'portal']][:20]

,item,orgao,portal
22901,"CABO DE EMBREAGEM PARA MOTO, PARA MOTOS DA MAR...",SECRETARIA DA SEGURANCA PUBLICA - CTO. SUPRIM....,BEC / Imprensa Oficial / Pregão SP
21401,"VENTOINHA DO RADIADOR PARA MOTO, MARCA MOTOS D...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,BEC / Imprensa Oficial / Pregão SP
21403,"KIT RELAÇÃO/TRANSMISSAO PARA MOTO, MARCA HONDA...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,BEC / Imprensa Oficial / Pregão SP
21400,"KIT RELAÇÃO/TRANSMISSAO PARA MOTO, MARCA YAMAH...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,BEC / Imprensa Oficial / Pregão SP
8089,"PLACA DE EMBREAGEM PARA MOTO, PARA MOTOS DA MA...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO DE P...,BEC / Imprensa Oficial / Pregão SP
22900,"RETENTOR DE OLEO DO GARFO PARA MOTO, PARA MOTO...",SECRETARIA DA SEGURANCA PUBLICA - CTO. SUPRIM....,BEC / Imprensa Oficial / Pregão SP
8088,"DISCO DE FRICCAO DE EMBREAGEM PARA MOTO, KIT C...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO DE P...,BEC / Imprensa Oficial / Pregão SP
21386,"PASTILHA DE FREIO PARA MOTO, POSICAO TRASEIRA,...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,BEC / Imprensa Oficial / Pregão SP
21389,"PASTILHA DE FREIO PARA MOTO, POSICAO TRASEIRA,...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,BEC / Imprensa Oficial / Pregão SP
21388,"PASTILHA DE FREIO PARA MOTO, POSICAO DIANTEIRA...",SECRETARIA DA SEGURANCA PUBLICA - COMANDO POLI...,BEC / Imprensa Oficial / Pregão SP
